[x] Utilize o dataset Titanic

[x] - Realize o pre-processamento dos dados: limpeza, tratamento de valores faltantes, conversão de dados categóricos vetorização, normalização.

[x] - Avalie um rede MLP utilizando o método de validação-cruzada com holdout. Varie o número de camadas ocultas (1,2,3) e de neurônios em cada camada oculta (16, 32,64). Use 10% do conjunto de treino para validação.

[x] - Use como função de ativação nas camadas ocultas a função ReLU e na camada de saída a softmax.

[x] - Para o treinamento utilize RMSprop como otimizador, entropia cruzada como função de loss (sparse_categorical_crossentropy). 

[x] - Treine utilizando parada antecidapa: EarlyStopping como callback. Veja exemplo. 

[x] - Monitore a acurácia durante o treino. Armazene as informações de treino usando history = model.fit(...). Imprima um gráfico que apresente a variação da acurácia de treino e de validação no decorrer das épocas. Imprima também um outro gráfico para a loss. Você encontra exemplos de como plotar essa informção aqui.

[x] - Avalie o melhor modelo com os dados de teste: model.evaluate(...)

In [1]:
import pandas as pd
train_data = pd.read_csv("./titanic/train.csv")
test_data = pd.read_csv("./titanic/test.csv")

train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

### Muitos valores da idade estão faltando, logo, tenho duas opções:
    [] Remover a coluna.
    [x] Colocar algum valor substituto, já que a idade na hora de salvar foi importante (Ex: crianças foram salvas primeiro).

In [5]:
def adicionar_idade(row):
    if pd.isnull(row["Age"]):
        return newAge
    else:
        return row["Age"]

newAge = train_data["Age"].median() # Pego a mediana das idades.
train_data['Age'] = train_data.apply(adicionar_idade, axis = 1)

newAge = test_data["Age"].median() # Pego a mediana das idades (Não a média).
test_data['Age'] = test_data.apply(adicionar_idade, axis = 1)

In [6]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

### Será que a cabine que a pessoa estava tem relação com sua sobrevivência?

In [8]:
cabin_survival = train_data.groupby("Cabin")["Survived"].mean()
print(cabin_survival)
# Percebemos que dependendo da Cabine a pessoa tem mais chances de sobreviver, algumas 100%, outras 0%...

Cabin
A10    0.0
A14    0.0
A16    1.0
A19    0.0
A20    1.0
      ... 
F33    1.0
F38    0.0
F4     1.0
G6     0.5
T      0.0
Name: Survived, Length: 147, dtype: float64


In [9]:
cabinNames = set(train_data['Cabin'])
mapping = [i for i in range(len(cabinNames))]
train_data['Cabin'].replace(cabinNames, mapping, inplace=True) # Mapeando para inteiro

cabinNames = set(test_data['Cabin'])
mapping = [i for i in range(len(cabinNames))]
test_data['Cabin'].replace(cabinNames, mapping, inplace=True) # Mapeando para inteiro

train_data['Cabin']

0       92
1      117
2       92
3       44
4       92
      ... 
886     92
887     66
888     92
889    146
890     92
Name: Cabin, Length: 891, dtype: int64

In [10]:
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

In [11]:
test_data.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin          0
Embarked       0
dtype: int64

### Lidando com o último atributo faltante (Fare)

In [12]:
fare_median = test_data["Fare"].median()
test_data["Fare"].fillna(fare_median, inplace=True) # Preenchendo valores nulos com a mediana

test_data.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### Será que o porto de embarque tem relação com sua sobrevivência?

In [13]:
embarked_survival = train_data.groupby("Embarked")["Survived"].mean()
print(embarked_survival)

Embarked
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


In [14]:
embarkedNames = set(train_data['Embarked'])
mapping = [i for i in range(len(embarkedNames))]
train_data['Embarked'].replace(embarkedNames, mapping, inplace=True) # Mapeando para inteiro

embarkedNames = set(test_data['Embarked'])
mapping = [i for i in range(len(embarkedNames))]
test_data['Embarked'].replace(embarkedNames, mapping, inplace=True) # Mapeando para inteiro

train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [15]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,92,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,117,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,92,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,44,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,92,1
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,92,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,4,1
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,92,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,92,1
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,92,3


### Convertendo atributo categórico Sex para numérico

In [16]:
train_data["Sex"].replace(["male", "female"], [0, 1], inplace=True)
test_data["Sex"].replace(["male", "female"], [0, 1], inplace=True)

train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,92,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,117,3
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,92,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,44,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,92,1
5,6,0,3,"Moran, Mr. James",0,28.0,0,0,330877,8.4583,92,0
6,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,4,1
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2.0,3,1,349909,21.0750,92,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,0,2,347742,11.1333,92,1
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,92,3


### Importando os dados

In [17]:
# Apagando colunas inúteis para a análise (Colunas como nome e Ticket, além de Survived já que é usada no y_train)
X_train = train_data.drop(['Survived', 'Name', 'Ticket','PassengerId'], axis=1)
# Pegando o y
y_train = train_data['Survived']

# Apagando colunas inúteis para a análise nos dados de teste
X_test = test_data.drop(['Name', 'Ticket','PassengerId'], axis=1)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Cabin     418 non-null    int64  
 7   Embarked  418 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 26.2 KB


### Normalizando dados de treino, validação e teste

In [18]:
from sklearn.preprocessing import StandardScaler

normalized = StandardScaler()
normalized.fit(X_train)

X_train_std = normalized.transform(X_train)
X_test_std = normalized.transform(X_test)

### Testando vários modelos

In [19]:
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

callback = EarlyStopping(monitor="val_loss", patience=20)

maxAcc = float("-inf")
numLayers = [1, 2, 3]
numNeurons = [16, 32, 64]

bestNumLayers = numLayers[0]
bestNumNeuron = numNeurons[0]


for nLayer in numLayers:
    for nNeuron in numNeurons:
        sqLayers = [Dense(nNeuron, activation="relu")] * (nLayer - 1) + [
            Dense(nNeuron, activation="softmax")
        ]
        model = Sequential(sqLayers)
        model.compile(
            optimizer="rmsprop",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy", "top_k_categorical_accuracy"],
        )
        history = model.fit(
            x=X_train_std,
            y=y_train,
            epochs=100,
            callbacks=[callback],
            validation_split=0.1,
            verbose=0,
        )

        if np.mean(history.history["val_accuracy"]) > maxAcc:
            bestNumLayers = nLayer
            bestNumNeuron = nNeuron

        print(f"Quantidade de camadas ocultas: {nLayer}")
        print(f"Quantidade de neurônios nesta camada: {nNeuron}\n\n")

        #  Variação da acurácia de treino e de validação
        plt.plot(history.history["accuracy"])
        plt.plot(history.history["val_accuracy"])
        plt.title("Acurácia do modelo")
        plt.ylabel("Acurácia")
        plt.xlabel("Época")
        plt.legend(["Treino", "Validação"], loc="bottom right")
        plt.show()

        # Variação do loss
        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title("Perdas do modelo")
        plt.ylabel("Perca")
        plt.xlabel("Época")
        plt.legend(["Treino", "Validação"], loc="bottom right")
        plt.show()

: 

: 

### Realizando a avaliação do modelo

In [ ]:
sqLayers = [Dense(bestNumNeuron, activation="relu")] * (bestNumLayers - 1) + [
    Dense(bestNumNeuron, activation="softmax")
]
model = Sequential(sqLayers)
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", "top_k_categorical_accuracy"],
)

model.fit(
    x=X_train_std,
    y=y_train,
    epochs=100,
    callbacks=[callback],
    validation_split=0.1,
    verbose=0,
)


y_pred = model.predict(X_test)
to_submit = pd.concat(
    [test_data.PassengerId, pd.DataFrame({"Survived": y_pred})], axis=1
)

to_submit.to_csv("output.csv", index = False)

to_submit

OBS: este programa gerará ao final um arquivo CSV que deverá ser utilizado na plataforma kaggle para avaliar a acurácia do modelo
